In [34]:
from time import time

num_parallel = 100
queries = ['allUris:"barackobama.com"'] * 100


tic = time()
make_queries(queries, parallel=num_parallel)
print("num. of parallel queries: {}, num. of seconds: {:.2f}".format(x, time()-tic))

http://kg.diffbot.com/kg/dql_endpoint
num. of parallel queries: 100, num. of seconds: 41.99
num. of parallel queries: 32, num. of seconds: 42.59
num. of parallel queries: 10, num. of seconds: 44.59
num. of parallel queries: 3, num. of seconds: 47.48
num. of parallel queries: 1, num. of seconds: 53.65


In [67]:
from diffbot_api import *

from sqlitedict import SqliteDict

CACHED_QUERY_DB = "diffbot-query-cache.sqlite"

class CachedQuery(object):
    def __init__(self, cache_fpath=CACHED_QUERY_DB):
        self._cache = SqliteDict(cache_fpath, autocommit=True)
        
    def __del__(self):
        self._cache.close()
        
    def make_query(self, query):
        if query in self._cache:
            return self._cache[query]
        else:
            response = make_query(query)
            self._cache[query] = make_query(query)
            return response
        
 
cq = CachedQuery()
%timeit r = cq.make_query('name:"Barack Obama"')
r.content

830 µs ± 120 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


b'{\n  "hits": 856,\n  "data": [\n    {\n      "types": ["Organization"],\n      "twitterUri": "/students-for-barack-obama",\n      "nbActiveEmployeeEdges": 25,\n      "isDissolved": true,\n      "importance": 59400,\n      "origin": "angel.co/students-for-barack-obama",\n      "diffbotUri": "http://diffbot.com/entity/OzeK877GbGZ",\n      "nbIncomingEdges": 594,\n      "nbEmployeesMin": 20,\n      "allUris": [\n        "angel.co/students-for-barack-obama",\n        "/students-for-barack-obama"\n      ],\n      "clusterId": "sgltn_angl_sentinel8410626",\n      "type": "Organization",\n      "allNames": ["Students for Barack Obama"],\n      "blogUri": "/students-for-barack-obama",\n      "angellistUri": "angel.co/students-for-barack-obama",\n      "name": "Students for Barack Obama",\n      "origins": ["angel.co/students-for-barack-obama"],\n      "nbEmployeesMax": 100,\n      "id": "OzeK877GbGZ",\n      "crawlTimestamp": 1488308400,\n      "linkedInUri": "/students-for-barack-obama",\n 

In [43]:
from ttl import *

phrases = "New York, windows, catwalk, teardrops, commuters, curved, floor, shape, video, walls, Grand Central Terminal"
phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
context = "Inside, it’s even wackier: curved walls, windows in the shape of teardrops, and a catwalk with a tiny video screen embedded in the floor that shows an endless loop of antlike commuters rushing through Grand Central Terminal in New York."
phrases

try:
    diffbot_cache

# def mfs(input_ttl):
# graph, context, phrases = parse_d2kb_ttl(input_ttl)

# print("# triples input:", len(graph))
for phrase in phrases:
    # get candidate wikipedia links from the Diffbot KB

    # name: phrase.text 
    
    # limit to the categories of interest
    
    # limit to the hits with wikipedia url 
    
    # sort by importance 
    
    print(phrase.text)
    
#     # save the results
#     graph.add( (phrase.subj, CLASS_URI, NONE_URI) )
#     graph.add( (phrase.subj, LINK_URI, NONE_URI) )
# print("# triples output:", len(graph))

# output_ttl = str(graph.serialize(format='n3', encoding="utf-8"), "utf-8")
#return output_ttl

New York
windows
catwalk
teardrops
commuters
curved
floor
shape
video
walls
Grand Central Terminal


In [35]:
russians

{'data': [{'allNames': ['रूसी लोग',
    'russos',
    'Rusi',
    'oroszok',
    'რუსეფი',
    'ռուս',
    'Lō͘-se-a-cho̍k',
    'Bangsa Rusia',
    'Roushie fowk',
    'Урусар',
    'Uruusiq',
    'Russe',
    'russi',
    'Rusos',
    'ruslar',
    '俄羅斯人',
    'Ngò̤-lò̤-sṳ̆-nè̤ng',
    'روس',
    'Vindläseq',
    'ГӀурусал',
    'Руш',
    'ロシア人',
    'Ruslar',
    'ruși',
    'Урыҫтар',
    'русские',
    'рускія',
    'Руснаци',
    'روسلار',
    'Рузт',
    'რუსები',
    '俄罗斯人',
    'Rusët',
    'Руси',
    'Орыстар',
    'Urang Rusia',
    'Оьрсий',
    'ryssar',
    '러시아인',
    'Rûs',
    'Орустар',
    'Rusové',
    'Роусьсци',
    'Вырăссем',
    'Русҳо',
    'ชาวรัสเซีย',
    'Russi',
    'Rwsiaid',
    'russere',
    'руслар',
    'Russen',
    'Расейцы',
    'Rusai',
    'Krievi',
    '𐍂𐌿𐍃𐌰𐌽𐍃',
    '俄国人',
    'Ngò-lò̤-sṳ̂-ngìn',
    'росіяни',
    'روسی',
    'Rosā',
    'Venälaižed',
    'Орос үндэстэн',
    'Орсин',
    'Ρώσοι',
    'Russians',
    'Rusoj',
    'ᎠᏂᏲᎾ',
  

In [19]:
%load_ext autoreload
%autoreload 2

# DQL

In [ ]:
query_and_save('allUris:"barackobama.com"', "data/all-uris.json")
query_and_save('wikipediaUri:"en.wikipedia.org/wiki/Barack_Obama"', "data/wiki-uri.json")
query_and_save('allUris:"en.wikipedia.org/wiki/Barack\_Obama"', "data/all-uris-wiki.json")
query_and_save('origins:"en.wikipedia.org/wiki/Barack_Obama"', "data/origins.json")

In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [ ]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")